In [1]:
%matplotlib inline
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from sklearn import svm
from sklearn.cross_validation import train_test_split
import glob
from hmmlearn.hmm import GaussianHMM, MultinomialHMM

/home/fernanda/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
datas_cer = glob.glob('features_projeto_IA369Y/treinamento/certeza/*.csv')
datas_inc = glob.glob('features_projeto_IA369Y/treinamento/incerteza/*.csv')
#datas_control = glob.glob('features_projeto_IA369Y/controles/*.csv')

In [3]:
def select_features(datas):
    feat_list = []
    for file in datas:
        df = pd.read_csv(file, sep=';')
        #df1 = df[df.columns[2:4]]
        df1 = df.loc[:, 'alphaRatio_sma3': 'F3amplitudeLogRelF0_sma3nz'].values
        d = np.array(df1)
        #print (d.shape)
        feat_list.append(d)
        lenghts = []
        for i in range(len(feat_list)):
            lenghts.append(len(feat_list[i]))
        #print (df1)
    f = np.vstack(feat_list)
    return f, lenghts, feat_list

In [4]:
f_cer, len_cer, list_fcer = select_features(datas_cer)
f_inc, len_inc, list_finc = select_features(datas_inc)
#f_control, len_control = select_features(datas_control)

#features = np.hstack(f_cer, f_inc)

print (len(len_cer))

26


In [5]:
import warnings
warnings.filterwarnings('ignore')

model1 = GaussianHMM(n_components=2, covariance_type='diag', n_iter=50)
#Parâmetros que determina a topologia left-right
#model1.startprob_ = np.array([1.0, 0.0, 0.0])#, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]) # len(stratprob) = n_components
#model1.transmat_ = np.array([[0.5, 0.5, 0.0],
                           # [0.0, 0.5, 0.5],
                           # [0.0, 0.0, 1.0]])
#model1.fit(f_control, len_cer)
model1.fit(f_cer, len_cer)

print (model1.monitor_)
print (model1.monitor_.converged)

ConvergenceMonitor(history=[-322985.65026263945, -322985.6500779627], iter=10,
          n_iter=50, tol=0.01, verbose=False)
True


In [6]:
model2 = GaussianHMM(n_components=2, covariance_type='diag', n_iter=50)
model2.fit(f_inc, len_inc)
print (model2.monitor_)
print (model2.monitor_.converged)

ConvergenceMonitor(history=[-338461.7431665952, -338461.743118681], iter=9,
          n_iter=50, tol=0.01, verbose=False)
True


## Etapa de validação dos dados

In [7]:
datas_v_cer = glob.glob('features_projeto_IA369Y/validacao/certeza/*.csv')
datas_v_inc = glob.glob('features_projeto_IA369Y/validacao/incerteza/*.csv')

v_cer, ven_cer, list_vcer = select_features(datas_v_cer)
v_inc, ven_inc, list_vinc = select_features(datas_v_inc)

#print (list_vcer[0].shape)
#print (len(list_vcer))
#print (ven_cer)
#print (v_cer.shape)

In [8]:
# Verificar a quantidade de acerto
# Para os áudios classificados como CERTEZA
def taxa_cer(list_vcer): #taxa de erro e acerto em porcentagem
    erros1 = []
    acertos1 = []
    for val in range(len(list_vcer)):
        prob1 = model1.score(list_vcer[val])
        prob2 = model2.score(list_vcer[val])
        if prob1 > prob2:
            acertos1.append(['certeza'])
        else:
            erros1.append(['incerteza'])
    taxa_ac1 = (len(acertos1))/len(list_vcer)*100
    taxa_er1 = (len(erros1))/len(list_vcer)*100
    print ('taxa de acerto:', taxa_ac1, 'taxa de erro:', taxa_er1)
    return taxa_ac1, taxa_er1

# Para os áudios classificados como INCERTEZA
def taxa_inc(list_vinc): #taxa de erro e acerto em porcentagem
    erros2 = []
    acertos2 = []
    for val2 in range(len(list_vinc)):
        prob3 = model1.score(list_vinc[val2])
        prob4 = model2.score(list_vinc[val2])
        if prob3 > prob4:
            erros2.append(['certeza'])
        else:
            acertos2.append(['incerteza'])
    taxa_ac2 = (len(acertos2))/len(list_vinc)*100
    taxa_er2 = (len(erros2))/len(list_vinc)*100
    print ('taxa de acerto:', taxa_ac2, 'taxa de erro:', taxa_er2)
    return taxa_ac2, taxa_er2


print ('Validação para certeza')
ac1, er1 = taxa_cer(list_vcer)

print ('Validação para incerteza')
ac2, er2 = taxa_inc(list_vinc)

Validação para certeza
taxa de acerto: 72.72727272727273 taxa de erro: 27.27272727272727
Validação para incerteza
taxa de acerto: 54.54545454545454 taxa de erro: 45.45454545454545


### Audio de teste

In [9]:
def select_feature_test(path=''):
    df = pd.read_csv(path, sep=';')
    df1 = df.loc[:, 'alphaRatio_sma3': 'F3amplitudeLogRelF0_sma3nz'].values
    d = np.array(df1)
    #print (d.shape)
    lenghts = []
    for i in range(len(d)):
        lenghts.append(len(d[i]))
    #print (df1)
    f = np.vstack(d)
    return f, lenghts

In [10]:
path_t = 'features_projeto_IA369Y/certeza/I22questao13.csv'
path_t2 = 'features_projeto_IA369Y/incerteza/I21questao11.csv'

f_test_cer, len_test_cer = select_feature_test(path_t)
f2_test_cer, len2_test_cer = select_feature_test(path_t2)

# Teste das perguntas de controle
#certeza
prob1 = model1.score(f_test_cer)
prob2 = model2.score(f_test_cer)
#print ('treino certeza e teste certeza:', prob1)
#print ('treino incerteza e teste certeza:', prob2)

#incerteza
prob3 = model1.score(f2_test_cer)
prob4 = model2.score(f2_test_cer)
#print ('treino incerteza e teste incerteza:', prob3)
#print ('treino incerteza e teste incerteza:', prob4)


In [11]:
#Incerteza
if prob3 > prob4:
    print ('certeza')
else:
    print ('incerteza')
    

incerteza


In [12]:
#Certeza
if prob1 > prob2:
    print ('certeza')
else:
    print ('incerteza')

certeza


In [13]:
# Utilizando o audio I22_questão1:
path1 = ('I22questao2.csv')
# Utilizando o audio I22_questão2:
#path2 = ('I22questao2.csv')

out_cer, l_cer = select_feature_test(path1)

prob_out_cer = model1.score(out_cer)
prob_out_inc = model2.score(out_cer)

#print (prob_out_cer)
#print (prob_out_inc)

if prob_out_cer > prob_out_inc:
    print ('certeza')
else:
    print ('incerteza')

incerteza
